In [20]:
import numpy as np 
import pandas as pd 
import bz2
import sentencepiece as spm
import random
import gc
import chardet
import re

import os
print(os.listdir("../input"))

['train.ft.txt.bz2', 'test.ft.txt.bz2']


In [21]:
train_file = bz2.BZ2File('../input/train.ft.txt.bz2')
test_file = bz2.BZ2File('../input/test.ft.txt.bz2')

**Create Lists containing Train & Test sentences**

In [22]:
train_file_lines = train_file.readlines()
test_file_lines = test_file.readlines()

**Convert from raw binary strings to strings that can be parsed**

In [23]:
train_file_lines = [x.decode('utf-8') for x in train_file_lines]
test_file_lines = [x.decode('utf-8') for x in test_file_lines]

**Check Data Appearance**

In [24]:
random.sample(train_file_lines, 10)

["__label__2 Dead On: Love the series. Hated to see the horse go down but I guess it provided a necessary distraction. Glad I have these to watch when I can't watch TV.\n",
 '__label__1 Intellectual Redundancy: When an author clearly aspires to be looked upon by the public eye as an "intellectual" and seeks to impress others with his extreem wordiness, redundancy of expressions and use of technical terms, he writes not for the benefit of all who will read his work but he writes to benefit his "intellectual" ego. Other than this obvious impression, the work is adequate enough for the subject.I was able to purchase this book for just over $5.00 and am not sure I paid too much.\n',
 "__label__1 Nonsense: This movie doesn't make a lot of sense. The motivation of the principal actor remains completely unclear up to the very end of the movie. In addition, he looks more like a US Marine than a Native American. The only notable performances come from Adam Beach and the congenial Raul Trujillo.

From the above output it can be seen that each sentence begins with it's sentiment **(__label__1 -> Negative, __label__2 -> Positive)**, which is then followed by the review and ends with a newline character **\n**. 

So, first I go convert all the labels to **O**(Negative) and **1**(Positive) and store it in lists that only contain the label values. After this, I store the remainder of the sentence excluding the newline character in lowercase in lists. Also, convert all numbers to **0**.

**Clean Data**

In [25]:
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_lines]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file_lines]

for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])
    
test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file_lines]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file_lines]

for i in range(len(test_sentences)):
    test_sentences[i] = re.sub('\d','0',test_sentences[i])

**Data before cleaning**

In [26]:
train_file_lines[70:75]

['__label__1 [Not what I expected]: ... My daughter and I loved the movie. She is 5 years old and it took her 2 hours to complete the game. Absolutely no challenge, and very little interaction. It is essentially just clicking on the screen to make it play the 7 or 8 scenes it has programmed. There are no games to play, nothing to figure out, nothing to do. Save your money.\n',
 '__label__1 barbie rapunzel = crying child: My 6-year old daughter is currently sobbing in her bedroom as a result of this rotten game.She spent an hour on this game painting in pretty pictures and coloring in flowers. But halfway through the game, the program hung, and wouldn\'t let us leave the room or click anywhere else. Now even if we exit the game and then start it back up, we are frozen in one spot while Barbie\'s disembodied voice urges us to "explore somewhere else in the castle."Standard software troubleshooting and rebooting didn\'t help. Vivendi Games\' support site is "currently disabled." And now I

**Data After Cleaning**

In [27]:
train_sentences[70:75]

['[not what i expected]: ... my daughter and i loved the movie. she is 0 years old and it took her 0 hours to complete the game. absolutely no challenge, and very little interaction. it is essentially just clicking on the screen to make it play the 0 or 0 scenes it has programmed. there are no games to play, nothing to figure out, nothing to do. save your money.',
 'barbie rapunzel = crying child: my 0-year old daughter is currently sobbing in her bedroom as a result of this rotten game.she spent an hour on this game painting in pretty pictures and coloring in flowers. but halfway through the game, the program hung, and wouldn\'t let us leave the room or click anywhere else. now even if we exit the game and then start it back up, we are frozen in one spot while barbie\'s disembodied voice urges us to "explore somewhere else in the castle."standard software troubleshooting and rebooting didn\'t help. vivendi games\' support site is "currently disabled." and now i\'m left with a weeping,

In [28]:
train_labels[70:75]

[0, 0, 1, 1, 0]

In [29]:
gc.collect()

1329

**TO-DO**

+Convert all the urls (eg: http, https, www etc) to < url >

In [30]:
for sentence in train_sentences[:1000]:
    if 'www' in sentence or 'http' in sentence:
        print(sentence)

why is this a good book?: as someone who has been practicing chinese medicine for 00 years in san diego (www.bodymindwellnesscenter.com), i find this book an excellent resource for anyone who wants and in dept primer into the theoretical basis and practical application of a system of medicine that originates in the chinese philosophy that emphasizes the natural balance found in nature and agriculture.
see my article "jean hill - the lady in red" online: since writing my review of jfk: the last dissenting witness in 0000, i have been able to have my 00-page article - "jean hill -the lady in red" - included in clint bradford's jfk assassination website. the address is: http://www.jfk-info.com/sitemap0.htm (scroll down for title).
it works, but....: my size medium was true to size. this made my waist smaller and while i had it on, i didn't eat as much because of the pressure, it makes me feel full; however, all of the cinchers do this as it is its purpose. i did like the fact that it's sh